In [8]:
pip install hazm

Note: you may need to restart the kernel to use updated packages.


In [9]:
from hazm import *


In [11]:
tagger = POSTagger(model='pos_tagger.model')
tagger.tag(word_tokenize('ما بسیار کتاب می‌خوانیم'))
[('ما', 'PRO'), ('بسیار', 'ADV'), ('کتاب', 'N'), ('می‌خوانیم', 'V')]


[('ما', 'PRO'), ('بسیار', 'ADV'), ('کتاب', 'N'), ('می\u200cخوانیم', 'V')]

In [ ]:
import pandas as pd
from hazm import Normalizer, word_tokenize, POSTagger

# ==== config (edit as needed) ====
IN_CSV   = "data/train.csv"            # must contain a 'persian' column (and optionally 'english')
OUT_CSV  = "data/train_pos.csv"
MODEL    = "pos_tagger.model"         # e.g. "resources/postagger.model"
DELIM    = ","                        # change if your CSV uses another delimiter
NROWS    = None                       # set to an int (e.g., 10) to sample only first N rows
# =================================

# load data
df = pd.read_csv(IN_CSV, delimiter=DELIM, nrows=NROWS)

# hazm components
tagger = POSTagger(model=MODEL)
norm = Normalizer(persian_numbers=True)

def pos_tags_only(text: str) -> list[str]:
    """Return POS tag sequence (no tokens), one tag per token."""
    if not isinstance(text, str) or not text.strip():
        return []
    # normalize + tokenize with the SAME toolchain you'll use elsewhere
    toks = word_tokenize(norm.normalize(text))
    tagged = tagger.tag(toks)              # [(tok, POS), ...]
    return [pos for _, pos in tagged]      # keep only POS

def tokenize(text: str) -> list[str]:
    if not isinstance(text, str) or not text.strip():
        return []
    return word_tokenize(norm.normalize(text))

# build columns
df["persian_tok"] = df["persian"].apply(tokenize)
df["persian_pos"] = df["persian"].apply(lambda s: " ".join(pos_tags_only(s)))

# (optional) quick alignment check
def count_ws(s): return len(str(s).split())
align_ok = []
bad_rows = []
for i, (sent, tags_str) in enumerate(zip(df["persian"], df["persian_pos"])):
    n_tok = len(tokenize(sent))
    n_tag = count_ws(tags_str)
    ok = (n_tok == n_tag)
    align_ok.append(ok)
    if not ok and len(bad_rows) < 10:
        bad_rows.append((i, n_tok, n_tag, sent, tags_str))

df["pos_align_ok"] = align_ok

# save
cols_to_save = [c for c in ["persian", "english", "persian_pos"] if c in df.columns]
df[cols_to_save].to_csv(OUT_CSV, index=False)
print(f"Saved -> {OUT_CSV}")
print(f"Alignment OK: {sum(align_ok)}/{len(align_ok)} "
      f"({100*sum(align_ok)/max(1,len(align_ok)):.1f}%)")

if bad_rows:
    print("\nFirst few misaligned rows (index, n_tok, n_tag):")
    for i, n_tok, n_tag, sent, tags in bad_rows:
        print(f"- row {i}: tokens={n_tok}, tags={n_tag}")


Saved -> data/pairs_pos_sample.csv
                                             persian  \
0  رئیس‌جمهور تاکید کرد: کار این دولت فقط با نتیج...   
1                          من دیگر تو را دوست ندارم.   
2                   ما یک خانه کامل در بلوک b داریم.   
3  عکس‌های خود را به pr@cuibul. com بفرستید یا در...   
4                 نه. این فقط شما را عالی‌تر می‌کند.   
5                      1396 / 12 / 9 12: 05: 56 ق. ظ   
6                                      (39 کیلوبایت)   
7               Herscher (Illinois) 815426 **** تلفن   
8  همه‌ی چیزایی که «چندلر» از روی. احساس گناه گرف...   
9                  11. موتور تون جایزه بزرگ (مسابقه)   

                                         persian_pos  
0  رئیس‌جمهور/NOUN تاکید/NOUN کرد/VERB :/PUNCT کا...  
1  من/PRON دیگر/ADV تو/PRON را/ADP دوست/NOUN ندار...  
2  ما/PRON یک/NUM خانه/NOUN,EZ کامل/ADJ در/ADP بل...  
3  عکس‌های/NOUN,EZ خود/PRON را/ADP به/ADP pr@cuib...  
4  نه/ADV ./PUNCT این/PRON فقط/ADV شما/PRON را/AD...  
5  ۱۳۹۶/NUM //PUNC